In [1]:
using Revise

In [2]:
using LazySets
using ModelVerification
using PyCall
using CSV
using ONNX
using Flux
using DataFrames

In [16]:
small_nnet_file = "../test/networks/small_nnet.nnet"
# small_nnet_file = net_path * "small_nnet.nnet"
# small_nnet encodes the simple function 24*max(x + 1.5, 0) + 18.5
small_nnet = read_nnet(small_nnet_file, last_layer_activation = ModelVerification.ReLU())
# model = Chain(
#     Dense(1, 2, relu),
#     Dense(2, 2, relu),
#     Dense(2, 1, relu))
flux_model = Flux.Chain(small_nnet)
println(flux_model.layers[1].weight, " ", flux_model.layers[1].bias) # max(x+1.5, 0) max(x+1.5, 0)
println(flux_model.layers[2].weight, " ", flux_model.layers[2].bias) # 4*max(x+1.5, 0)+2.5 4*max(x+1.5, 0)+2.5
println(flux_model.layers[3].weight, " ", flux_model.layers[3].bias) # 24*max(x+1.5, 0)+18.5
# in_hyper  = Hyperrectangle(low = [-0.9], high = [0.9])
in_hyper  = Hyperrectangle(low = [-2.5], high = [2.5]) # expected: [18.5, 114.5]
out_violated    = Hyperrectangle(low = [19], high = [114]) # 20.0 ≤ y ≤ 90.0
out_holds = Hyperrectangle(low = [18], high = [115.0]) # -1.0 ≤ y ≤ 50.0
info = nothing
search_method = BFS(max_iter=1, batch_size=13)
split_method = Bisect(1)
prop_method = Crown()
problem = Problem(flux_model, in_hyper, out_holds)
res = verify(search_method, split_method, prop_method, problem)
println(res)

[1.0; 1.0;;] 

[1.5, 1.5]
[2.0 2.0; 2.0 2.0] [2.5, 2.5]
[3.0 3.0] [3.5]
forward
in
out
false
forward
in
out
false
forward
in
out
false
BasicResult(:violated)
